# Rag application with LangChain

This step will define the agent which connects the embedding model, the chat model and the vector store to fullfill the RAG scenario

## Project Initialization

Initialize a DigitalHub project using consistent naming with other tutorials.


In [ ]:
import digitalhub as dh
import getpass as gt

USERNAME = gt.getuser()

project = dh.get_or_create_project(f"{USERNAME}-tutorial-project")
print(project.name)

## 1. Fetch components 

Let's get the components definition. We are looking for the latest RUNNING run for the 2 functions.

In [ ]:
embed_function = project.get_function("embed")
embed_run = embed_function.list_runs()[0]
if embed_run:
    print(f"Found run {embed_run.id}")

In [ ]:
llm_function = project.get_function("llama32-1b")
llm_run = llm_function.list_runs()[0]
if llm_run:
    print(f"Found run {llm_run.id}")

In [ ]:
EMBED_URL = embed_run.status.to_dict()["service"]["url"]
EMBED_MODEL = embed_run.status.to_dict()["openai"]["model"]
print(f"service {EMBED_URL} with model {EMBED_MODEL}")

In [ ]:
CHAT_URL = llm_run.status.to_dict()["service"]["url"]
CHAT_MODEL = llm_run.status.to_dict()["openai"]["model"]
print(f"service {CHAT_URL} with model {CHAT_MODEL}")

## 2. Create the agent 

We'll register a python function implementing the RAG agent with LangChain

In [ ]:
serve_func = project.new_function(
    name="rag-service", 
    kind="python", 
    python_version="PYTHON3_10", 
    image="harbor.digitalhub.smartcommunitylab.it/dslab/dslab-platform-harbor.atlas.fbk.eu/dslab/dslab-platform-msaloni-tutorial-project-rag-service:bc27b",
    code_src="src/serve.py",     
    handler="serve",
    init_function="init",
    requirements = [
        "transformers==4.50.3",
        "psycopg[binary]==3.2.10",
        "openai==1.109.1",
        "langchain==0.3.27",
        "langchain-core==0.3.76",
        "langchain-openai==0.3.27",
        "langchain-postgres==0.0.15",
        "langchain-huggingface==0.3.1",
        "langgraph==0.6.7",
        "filelock==3.19.1",
        "huggingface-hub==0.35.1",
        "beautifulsoup4>=4.13.0",
        "langchainhub==0.1.21"
    ]
)

And then we can run an instance connecting the model services together.

In [ ]:
serve_run = serve_func.run(
    action="serve",
    resources={
        "mem":  "8Gi",
    },
    envs=[
            {"name": "CHAT_MODEL_NAME", "value": CHAT_MODEL},
            {"name": "CHAT_SERVICE_URL", "value": CHAT_URL},
            {"name": "EMBEDDING_MODEL_NAME", "value": EMBED_MODEL},
            {"name": "EMBEDDING_SERVICE_URL", "value": EMBED_URL}
         ],
    secrets=["PG_CONN_URL"],
    wait=True
)

If the execution fails, it is probably due to the large number of dependencies requirted. We need to build an image.

In [ ]:
#serve_func.run("build",wait=True)

To test our API we make a call to the service endpoint providing a JSON with an example question.

In [ ]:
AGENT_URL = serve_run.status.to_dict()["service"]["url"]
print(AGENT_URL)


In [ ]:
import requests
import pprint
pp = pprint.PrettyPrinter(indent=2)

res = requests.post(f"http://{AGENT_URL}",json={"question": "how can i use a personal access token DigitalHub console?"})
pp.pprint(res.json())